In [0]:
%sql
use catalog basab_catalog_retail;
use schema gold_tables;

In [0]:
%sql
select * from day_wise_sale

In [0]:
%sql
select store_id,total_revenue from pos_stores_summary
order by total_revenue desc

In [0]:
%sql
select * from daily_sales limit 3

In [0]:
pos_df=spark.read.table('silver_tables.pos_silver')

In [0]:
pos_df.limit(2).display()

In [0]:
products_df=spark.read.table('silver_tables.products_silver')

In [0]:
pos_join_products = (
    pos_df.join(products_df, pos_df.sku_id == products_df.sku_id, "inner")
          .drop(products_df.sku_id)   # 👈 keeps pos_df.sku_id
)

In [0]:
pos_join_products.limit(3).display()

In [0]:
pos_join_products.write.format('delta').mode('overwrite').saveAsTable('silver_tables.pos_join_products')

In [0]:
%sql
create or replace table basab_catalog_retail.gold_tables.store_category_revenue as

select store_id,category,round(sum(units_sold*price),2) as total_category_revenue
from basab_catalog_retail.silver_tables.pos_join_products
where category != 'Unknown'
group by store_id,category
order by store_id,total_category_revenue desc


In [0]:
store_cat_rev_df=spark.table('gold_tables.store_category_revenue')

In [0]:
store_cat_rev_df.display()

In [0]:
inventory_df=spark.table('silver_tables.inventory_silver')

In [0]:
inventory_join_products=inventory_df.join(products_df, inventory_df.sku_id == products_df.sku_id, "inner").drop("products_df.sku_id")

In [0]:
inventory_join_products.display()

In [0]:
inventory_join_products.printSchema()


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
inventory_summary = (
    inventory_join_products
    .groupBy("store_id", "category")
    .agg(sum("stock_level").alias("total_stock_level")).orderBy("store_id")
)
inventory_summary.display()

In [0]:


inventory_summary2 = inventory_summary.filter(col("category") != "Unknown")


In [0]:
inventory_summary2.orderBy("store_id", desc("total_stock_level")).display()

In [0]:
store_cat_rev_df.display()

In [0]:
store_cat_full_df = store_cat_rev_df.join(
    inventory_summary2,
    on=["store_id", "category"],
    how="left"   # Use left join to keep all revenue rows even if stock is missing
)

store_cat_full_df.display()

In [0]:
store_cat_full_df2=store_cat_full_df.fillna({
    "total_stock_level": 0
})



In [0]:
store_cat_full_df2.display()

In [0]:
store_cat_full_df2.write.format('delta').mode('overwrite').saveAsTable('gold_tables.Store_Category_Report')

In [0]:
threshold = 500  # for example, if stock < 500 → High

store_cat_full_df3 = store_cat_full_df2.withColumn(
    "replenish_needed",
    when(col("total_stock_level") < threshold, "High")
    .otherwise("Low")
)

store_cat_full_df3.display()

In [0]:
store_cat_full_df3.filter(col("replenish_needed") == "High").display()

In [0]:
%sql
select * from basab_catalog_retail.gold_tables.region_wise_sales

In [0]:
%sql
select * from basab_catalog_retail.gold_tables.day_wise_sale

In [0]:
%sql
select * from basab_catalog_retail.gold_tables.storeformat_wise_unitsells

In [0]:
%sql
select * from basab_catalog_retail.gold_tables.pos_stores_summary